In [49]:
using StatsPlots, Statistics, Distributions, Random, PyPlot

In [59]:
n_bins = 10
n_RVs = 100000

mean_sum = 0
var_sum = 0

RV_list = Array{Distribution}(undef, n_RVs)

for i = 1:n_RVs
    μ = rand(5:10)
    σ = rand(1:6)
    
    temp_RV = Normal(μ, σ)
    RV_list[i] = temp_RV
    
    mean_sum = mean_sum + μ
    var_sum = var_sum + σ^2
end


In [60]:
function generate_midpoints(n_bins)
    quantiles = Array{Float64}(undef, n_bins)

    #generate the quantile midpoints
    for i = 1:n_bins
        quantiles[i] = i/n_bins - (1/2)*(1/n_bins)
    end

    return vec(quantiles)
end

generate_midpoints (generic function with 1 method)

In [61]:
function add_distributions(list1, list2)
    sum_list = Array{Float64}(undef, length(list1)^2)
    
    list1 = repeat(list1, length(list1))
    list1 = sort(list1)
    list2 = repeat(list2, length(list2))
    
    for i in eachindex(sum_list)
        sum_list[i] = list1[i] + list2[i]
    end
    
    return vec(sort(sum_list))
end

add_distributions (generic function with 1 method)

In [62]:
function re_bin(distribution, n_bins)
    rebinned = reshape(distribution, n_bins, n_bins)
    rebinned = mean(rebinned, dims = 1)
    
    return vec(rebinned)
end

re_bin (generic function with 1 method)

In [63]:
quantile_boundaries = generate_midpoints(n_bins)
println(quantile_boundaries)

final_RV = pop!(RV_list)
final_RV = quantile(final_RV, quantile_boundaries)
println(final_RV)

for RV in RV_list
    temp_list = quantile(RV, quantile_boundaries)
    
    final_RV = add_distributions(temp_list, final_RV)
    final_RV = re_bin(final_RV, n_bins) 
end


[0.05, 0.15000000000000002, 0.25, 0.35000000000000003, 0.45, 0.5499999999999999, 0.6499999999999999, 0.75, 0.85, 0.95]
[2.065439119145582, 3.8906998315186314, 4.976530749411754, 5.8440386007772975, 6.6230159594347775, 7.376984040565222, 8.155961399222702, 9.023469250588246, 10.109300168481369, 11.934560880854416]


In [64]:
println("Expected Mean is: ", mean_sum)
println("Calculated mean is: ", mean(final_RV))

Expected Mean is: 749848
Calculated mean is: 749848.0000000014


In [65]:
final_dist = fit(Normal, final_RV)
println(final_dist)

true_mu = mean_sum
true_sigma = var_sum^0.5
println(var_sum)
println(true_sigma)

true_dist = Normal(mean_sum, var_sum^(1/2))

Normal{Float64}(μ=749848.0000000014, σ=56.6937882260151)
1512145
1229.6930511310536


Normal{Float64}(μ=749848.0, σ=1229.6930511310536)